In [2]:
dataset_base_path = "/media/jfallmann/T9/University/master_thesis/dataset"
raw_snp_path = f"{dataset_base_path}/snp/raw"
snp_path = f"{dataset_base_path}/snp"
processed_snps = f"{snp_path}/processed"

In [3]:
import pandas as pd

#snp data taken from https://advp.niagads.org/
variants = pd.read_csv(f"{snp_path}/advp_variants.tsv", sep='\t')
variants.head()

,#dbSNP_hg38_chr,dbSNP_hg38_position,Top SNP,P-value,LocusName,RA 1(Reported Allele 1),nonref_allele,nonref_effect,OR_nonref,nearest_gene_symb,Study type,Study Design,Pubmed PMID,Population_map,Cohort_simple3,Sample size,Analysis group,Phenotype,Phenotype-derived,most_severe_consequence
0,chr1,6434683,rs12074379,0.00726,ESPN,T,T,NR,NaN,ESPN,SNP-based,Disease risk,30636644,Caucasian,"ADGC, CHS, CHARGE, HRS",10191,Plan 3 (only females),AD,AD,intron_variant
1,chr1,6434683,rs12074379,8.51E-40,NR,T,T,NR,NaN,ESPN,SNP-based,eQTL,30636644,Caucasian,"ADGC, CHS, CHARGE, HRS",10191,Plan 3 (only females),ESPN (ILMN_1806710) expression,Expression,intron_variant
2,chr1,8708071,rs112053331,0.0009,RERE,NR,NR,NR,NaN,RERE,SNP-based,Cross phenotype,30010129,Caucasian,IGAP,54162,All,AD,AD,intron_variant
3,chr1,8708071,rs112053331,0.08392,NaN,NR,NR,NR,NaN,RERE,Gene-based,Cross phenotype,30010129,Caucasian,IGAP,54162,All,AD,AD,intron_variant
4,chr1,11487007,rs2379135,0.0156,PTCHD2,NR,NR,NR,NaN,DISP3,SNP-based,Endophenotype,22245343,Caucasian,ADNI,757,All,MRI,Imaging,intron_variant


In [4]:
# order variants by p-value and select the top 150 variants
top_variants = variants.sort_values('P-value')
top_variants

,#dbSNP_hg38_chr,dbSNP_hg38_position,Top SNP,P-value,LocusName,RA 1(Reported Allele 1),nonref_allele,nonref_effect,OR_nonref,nearest_gene_symb,Study type,Study Design,Pubmed PMID,Population_map,Cohort_simple3,Sample size,Analysis group,Phenotype,Phenotype-derived,most_severe_consequence
2206,chr14,73809286,rs17182607,0,NR,A,A,NR,NaN,MIDEAS,SNP-based,eQTL,31055733,Caucasian,"ADGC, CHS, CHARGE, HRS",4806,HTN-N group,LOC100506476 (ENSG00000259065) expression,Expression,intergenic_region
3374,chr19,44892362,rs2075650,0,TOMM40,NR,NR,NR,NaN,TOMM40,SNP-based,Cross phenotype,28870582,Caucasian,IGAP,54162,All,AD,AD,intron_variant
1182,chr11,49980436,rs11602981,0,NR,C,C,NR,NaN,OR4C12,SNP-based,eQTL,31055733,Caucasian,"ADGC, CHS, CHARGE, HRS",4806,HTN-N group,FOLH1 (ENSG00000086205) expression,Expression,downstream_gene_variant
4870,chr4,173173789,rs62341097,0.000000006,GALNT7,A,A,-1.15,0.316637,GALNT7,SNP-based,Endophenotype,25188341,Caucasian,ADGC,4046,All,Neuritic Plaque,Neuropathology,intron_variant
1886,chr11,132065380,rs1629316,0.000000008,HNT,NaN,NaN,NaN,NaN,NTM,SNP-based,Endophenotype,17564960,genetically isolated community in the southwes...,GRIP,197,All,Cognitive function (Memory - Learning (or work...,Cognitive,intron_variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4629,chr3,153652877,rs11711889,NaN,NR,NR,NR,NR,NaN,ARHGEF26,SNP-based,Disease risk,21059989,Caucasian,ADGC,2449,Family-based individuals,LOAD,AD,intron_variant
5418,chr7,10277135,rs10271466,NaN,NR,NR,NR,NR,NaN,NDUFA4,SNP-based,Disease risk,21059989,Caucasian,ADGC,2449,Family-based individuals,LOAD,AD,intergenic_region
5419,chr7,10277135,rs10271466,NaN,NR,NR,NR,NR,NaN,NDUFA4,SNP-based,Disease risk,21059989,Caucasian,ADGC,2449,Family-based individuals,LOAD,AD,intergenic_region
6163,chr9,8095638,rs6477258,NaN,NR,NR,NR,NR,NaN,DMAC1,SNP-based,Disease risk,21059989,Caucasian,ADGC,2449,Family-based individuals,LOAD,AD,intergenic_region


In [22]:
top_k_snps = 1024
top_snps = top_variants['Top SNP'].unique()[:top_k_snps]
top_snps = set([str(snp).lower() for snp in top_snps])
print(f"Selected {len(top_snps)} top snps, including: {list(top_snps)[:5]}")

Selected 1024 top snps, including: ['rs1736556', 'rs10911736', 'rs676768', 'rs12422883', 'rs1887726']


In [7]:
MISSING_CALL = 0
REFERENCE = 1
HETEROZYGOUS_MUTANT = 2
HOMOZYGOUS_MUTANT = 3

In [8]:
class PatientGenome:
    def __init__(self, snps: list, subject_id: str):
        # we store a map with snp-id to type and init all snps with missing call
        self.genome = {snp: MISSING_CALL for snp in snps}
        self.subject_id = subject_id

    def add_genome(self, snp_id, snp_type):
        self.genome[snp_id] = snp_type

In [35]:
# load relevant subjects
subjects = []

with open(f'{dataset_base_path}/subjects.txt') as subjects_file:
    for line in subjects_file:
        subjects.append(line.strip())

In [10]:
def clean_id(id: str) -> str:
    return id.replace("_", "").replace(".", "").replace("-", "").lower()

In [11]:
import csv
import os
import re
import glob
from tqdm.notebook import tqdm

genome_data = []
n_worker = 8

def get_genotype_with_highest_quality(samples):
    highest_quality = -1
    highest_quality_sample = None
    for sample in samples:
        sample_data = sample.split(':')
        quality = sample_data[1]
        quality_numeric = int(quality)
        if quality_numeric > highest_quality:
            highest_quality = quality_numeric
            highest_quality_sample = sample
    return highest_quality_sample


def get_snps_from_vcf(vcf_file, top_snps):
    genome_split_regex = r'(.*)_SNPs'
    subject_id = re.search(genome_split_regex, os.path.basename(vcf_file)).group(1)
    subject_id = clean_id(subject_id)
    if subject_id not in subjects:
        return None
    genome = PatientGenome(top_snps, subject_id)
    # read vcf file
    with open(vcf_file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if row[0].startswith("#"):
                continue
            chrom, pos, id, ref, alt, qual, filter, info, format, *samples = row

            if id.lower() not in top_snps:
                continue

            # select the sample with the highest quality score
            sample = get_genotype_with_highest_quality(samples)
            sample_data = sample.split(':')
            genotype = sample_data[0]
            if genotype == '0/0':
                genome.add_genome(id, REFERENCE)
            elif genotype == '0/1':
                genome.add_genome(id, HETEROZYGOUS_MUTANT)
            elif genotype == '1/1':
                genome.add_genome(id, HOMOZYGOUS_MUTANT)
            else:
                genome.add_genome(id, MISSING_CALL)
    return genome


vcf_files = glob.glob(f"{raw_snp_path}/**/*.vcf")

for vcf_file in tqdm(vcf_files):
    genome = get_snps_from_vcf(vcf_file, top_snps)
    if genome is not None:
        genome_data.append(genome)

  0%|          | 0/809 [00:00<?, ?it/s]

In [36]:
# remove all genome data where subject is not in subjects
genome_data = [genome for genome in genome_data if genome.subject_id in subjects]
len(genome_data)

409

In [37]:
# remove all keys of all genomes not in top_snps
for genome in genome_data:
    for key in list(genome.genome.keys()):
        if key not in top_snps:
            del genome.genome[key]

In [38]:
# get average number of missing calls
missing_calls = []
for genome in genome_data:
    missing_calls.append(list(genome.genome.values()).count(MISSING_CALL))

avg = sum(missing_calls) / len(missing_calls)
std = (sum((x - avg) ** 2 for x in missing_calls) / len(missing_calls)) ** 0.5
avg, std

(666.8606356968215, 15.590657367503841)

In [39]:
# any genome with less than 1024 snps?
for genome in genome_data:
    if len(genome.genome) < 1024:
        print(genome.subject_id)

In [40]:
# create an array ordered by subjects and columns being the snp values in the same order as top_snps
import numpy as np

genomes = np.zeros((len(genome_data), len(top_snps)), dtype=np.int8)

# sort genome in same order as subjects list
genome_data.sort(key=lambda x: subjects.index(x.subject_id))

for i, genome in enumerate(genome_data):
    for j, snp in enumerate(top_snps):
        genomes[i, j] = genome.genome[snp]

In [41]:
# create folder for processed snps
if not os.path.exists(processed_snps):
    os.makedirs(processed_snps)

# store list of snp names to file
with open(f"{processed_snps}/snp_names.txt", 'w') as f:
    for snp in top_snps:
        f.write(f"{snp}\n")

# store the genomes to file
np.save(f"{processed_snps}/genomes.npy", genomes)

In [5]:
# load the genomes
import numpy as np
genomes = np.load(f"{processed_snps}/genomes.npy")


In [6]:
genomes

array([[2, 0, 3, ..., 3, 0, 0],
       [2, 0, 2, ..., 3, 0, 0],
       [3, 3, 3, ..., 3, 0, 0],
       ...,
       [2, 3, 3, ..., 3, 3, 0],
       [2, 0, 3, ..., 3, 0, 0],
       [2, 0, 3, ..., 3, 0, 0]], dtype=int8)

In [ ]:
# the rows of the genomes array are the subjects and the columns are the snps
# we want to

In [34]:
# is there a column with only missing calls?
missing_calls = []
for i in range(genomes.shape[1]):
    missing_calls.append(list(genomes[:, i]).count(MISSING_CALL))

missing_calls

[79,
 366,
 9,
 407,
 128,
 387,
 63,
 211,
 74,
 371,
 214,
 409,
 246,
 117,
 322,
 333,
 312,
 309,
 347,
 10,
 312,
 404,
 384,
 181,
 23,
 195,
 365,
 64,
 49,
 355,
 326,
 34,
 314,
 212,
 102,
 344,
 35,
 49,
 287,
 132,
 29,
 305,
 311,
 366,
 136,
 389,
 194,
 400,
 374,
 306,
 219,
 356,
 389,
 353,
 241,
 329,
 254,
 182,
 394,
 301,
 390,
 282,
 215,
 353,
 330,
 348,
 377,
 402,
 146,
 157,
 216,
 208,
 272,
 365,
 397,
 315,
 40,
 104,
 162,
 410,
 406,
 393,
 67,
 355,
 62,
 62,
 398,
 400,
 178,
 154,
 407,
 69,
 322,
 388,
 388,
 394,
 119,
 120,
 346,
 152,
 404,
 194,
 82,
 76,
 84,
 370,
 330,
 80,
 333,
 387,
 343,
 283,
 156,
 74,
 376,
 154,
 275,
 359,
 110,
 6,
 356,
 136,
 409,
 273,
 273,
 349,
 402,
 102,
 178,
 327,
 382,
 282,
 335,
 409,
 171,
 410,
 381,
 358,
 237,
 309,
 400,
 382,
 382,
 393,
 365,
 409,
 110,
 365,
 341,
 236,
 283,
 147,
 89,
 398,
 377,
 410,
 78,
 346,
 243,
 271,
 346,
 175,
 180,
 330,
 7,
 405,
 389,
 318,
 156,
 409,
 371,
 10